# StyleGAN2 -> Style-Mixing

***AmirHossein Bayat & Melika Emami***

This notebook demonstrates how to run NVIDIA's StyleGAN2 on Google Colab to do Style Mixing With your custom photo.
Make sure to specify a GPU runtime.

For information on StyleGAN2, see:

Paper: https://arxiv.org/abs/1812.04948

Video: https://youtu.be/kSLJriaOumA

Code: https://github.com/NVlabs/stylegan

FFHQ: https://github.com/NVlabs/ffhq-dataset

/Mikael Christensen, 2019.


In [0]:
%tensorflow_version 1.x
import tensorflow as tf

# Download the code
!git clone https://github.com/NVlabs/stylegan2.git
%cd stylegan2
!nvcc test_nvcc.cu -o test_nvcc -run

print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

Cloning into 'stylegan2'...
remote: Enumerating objects: 93, done.
remote: Total 93 (delta 0), reused 0 (delta 0), pack-reused 93
Unpacking objects: 100% (93/93), done.
/content/stylegan2
CPU says hello.
GPU says hello.
Tensorflow version: 1.15.0
GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-db91432b-5bb8-0555-f370-71f884bb181b)
GPU Identified at: /device:GPU:0


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Required Files**

Now you need to add below files to your google drive. Make sure you change path of files if needed.

In [0]:
# Projector files to convert given photos to dlatent vectors. I've changed a few things in original files from StyleGAN2, so you need to upload these files.
!cp "/content/drive/My Drive/StyleGAN2/run_projector.py" "/content/stylegan2/run_projector.py"
!cp "/content/drive/My Drive/StyleGAN2/projector.py" "/content/stylegan2/projector.py"
# These files are used to align the given pictures and export the face part from it.
!mkdir ffhq_dataset
!cp "/content/drive/My Drive/StyleGAN2/ffhq_dataset/__init__.py" "/content/stylegan2/ffhq_dataset/__init__.py"
!cp "/content/drive/My Drive/StyleGAN2/ffhq_dataset/face_alignment.py" "/content/stylegan2/ffhq_dataset/face_alignment.py"
!cp "/content/drive/My Drive/StyleGAN2/ffhq_dataset/landmarks_detector.py" "/content/stylegan2/ffhq_dataset/landmarks_detector.py"
!cp "/content/drive/My Drive/StyleGAN2/align_images.py" "/content/stylegan2/align_images.py"
# In this part you have to specify your custom pictures to be used. I've named my pictures like below.
!mkdir Pictures
!cp "/content/drive/My Drive/StyleGAN2/Pic1.JPG" "/content/stylegan2/Pictures/Pic1.JPG"
!cp "/content/drive/My Drive/StyleGAN2/Pic2.JPG" "/content/stylegan2/Pictures/Pic2.JPG"
!cp "/content/drive/My Drive/StyleGAN2/Pic3.JPG" "/content/stylegan2/Pictures/Pic3.JPG"
!cp "/content/drive/My Drive/StyleGAN2/Pic4.jpg" "/content/stylegan2/Pictures/Pic4.JPG"
# Make a folder to save aligned pictures in it.
!mkdir aligned_pictures

'!mkdir ffhq_dataset\n!cp "/content/drive/My Drive/StyleGAN2/ffhq_dataset/__init__.py" "/content/stylegan2/ffhq_dataset/__init__.py"\n!cp "/content/drive/My Drive/StyleGAN2/ffhq_dataset/face_alignment.py" "/content/stylegan2/ffhq_dataset/face_alignment.py"\n!cp "/content/drive/My Drive/StyleGAN2/ffhq_dataset/landmarks_detector.py" "/content/stylegan2/ffhq_dataset/landmarks_detector.py"\n!cp "/content/drive/My Drive/StyleGAN2/align_images.py" "/content/stylegan2/align_images.py"\n!mkdir Pictures\n!cp "/content/drive/My Drive/StyleGAN2/Pic1.JPG" "/content/stylegan2/Pictures/Pic1.JPG"\n!cp "/content/drive/My Drive/StyleGAN2/Pic2.JPG" "/content/stylegan2/Pictures/Pic2.JPG"\n!cp "/content/drive/My Drive/StyleGAN2/Pic3.JPG" "/content/stylegan2/Pictures/Pic3.JPG"\n!cp "/content/drive/My Drive/StyleGAN2/Pic4.jpg" "/content/stylegan2/Pictures/Pic4.JPG"\n!mkdir aligned_pictures'

**Aligning Pictures**

Now you need to run the align_images.py to do the aligning. Parameters are the folder containing your pictures and the folder to save the aligned pictures.

In [0]:
%run align_images.py Pictures/ aligned_pictures/

Using TensorFlow backend.


64045056/64040097 [==============================] - 173s 3us/step


**Generate .tfrecords**

We use the dataset_tool.py from StyleGAN2 to convert aligned pictures to tfrecords files.

In [0]:
%run dataset_tool.py create_from_images Datasets aligned_pictures

Loading images from "aligned_pictures"
Creating dataset "Datasets"
Added 4 images.


**Projecting pictures**

In this part we use our customized run_projector.py file to the image projection. We set the network parameter to StyleGAN2-ffhq-config-f and Project only one of our converted pictures. (You can change the --num-images to whatever you want. The default step parameter for this part is set to 1500 which you can change on **projector.py** file. After running this part some .npy files containing dlatents will generate in results folder related to each picture.(There's a little bug here which does not affect our work.)

In [0]:
%run run_projector.py project-real-images --network=gdrive:networks/stylegan2-ffhq-config-f.pkl --dataset=Datasets --data-dir=. --num-images=1

**Style Mixing Function**

In this function we load the network and use it to map some given seed to dlatent vectors as **Style**. The dlatent parameter is the loaded .npy files which we generated in the previous step.

In [0]:
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys

import pretrained_networks

def style_mixing(dlatent, col_seeds, truncation_psi, col_styles, minibatch_size=4):
    print('Loading networks from ...')
    _G, _D, Gs = pretrained_networks.load_networks('gdrive:networks/stylegan2-ffhq-config-f.pkl')
    w_avg = Gs.get_var('dlatent_avg') # [component]
    row_seeds = [80]
    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = False
    Gs_syn_kwargs.minibatch_size = 1 #minibatch_size

    print('Generating W vectors...')
    all_seeds = list(set(col_seeds))
    all_z = np.stack([np.random.RandomState(seed).randn(*Gs.input_shape[1:]) for seed in all_seeds]) # [minibatch, component]

    all_w = Gs.components.mapping.run(all_z, None) # [minibatch, layer, component]
    all_w = np.concatenate((all_w, dlatent), 0)
    all_w = w_avg + (all_w - w_avg) * truncation_psi # [minibatch, layer, component]
    all_seeds.extend(row_seeds) #= list(set(col_seeds + row_seeds))
    w_dict = {seed: w for seed, w in zip(all_seeds, list(all_w))} # [layer, component]
    
    print('Generating images...')
    all_images = Gs.components.synthesis.run(all_w, **Gs_syn_kwargs) # [minibatch, height, width, channel]
    image_dict = {(seed, seed): image for seed, image in zip(all_seeds, list(all_images))}

    print('Generating style-mixed images...')
    for row_seed in row_seeds:
        for col_seed in col_seeds:
            w = w_dict[row_seed].copy()
            # We set col_styles in this part
            w[3:8] = w_dict[col_seed][3:8]
            image = Gs.components.synthesis.run(w[np.newaxis], **Gs_syn_kwargs)[0]
            image_dict[(row_seed, col_seed)] = image

    print('Saving images...')
    for (row_seed, col_seed), image in image_dict.items():
        PIL.Image.fromarray(image, 'RGB').save(dnnlib.make_run_dir_path('Mix/'+'%d-%d.png' % (row_seed, col_seed)))

    print('Saving image grid...')
    _N, _C, H, W = Gs.output_shape
    canvas = PIL.Image.new('RGB', (W * (len(col_seeds) + 1), H * (len(row_seeds) + 1)), 'black')
    for row_idx, row_seed in enumerate([None] + row_seeds):
        for col_idx, col_seed in enumerate([None] + col_seeds):
            if row_seed is None and col_seed is None:
                continue
            key = (row_seed, col_seed)
            if row_seed is None:
                key = (col_seed, col_seed)
            if col_seed is None:
                key = (row_seed, row_seed)
            canvas.paste(PIL.Image.fromarray(image_dict[key], 'RGB'), (W * col_idx, H * row_idx))
    canvas.save(dnnlib.make_run_dir_path('Mix/grid.png'))

Load some dlatent.npy file and pass it to function.

In [0]:
dlat = np.load('/content/stylegan2/results/00002-project-real-images/image0000--dlatent.npy')
style_mixing(dlat, [44,55,1000,10,3,100,75,458,1500], 0.5, 5)

If you want to zip results folder to download it run the following cell.

In [0]:
import shutil

zip_name = '/content/stylegan2/resultszip'
directory_name = '/content/stylegan2/results'

# Create 'path\to\zip_file.zip'
shutil.make_archive(zip_name, 'zip', directory_name)

'/content/stylegan2/resultszip.zip'